In [ ]:
from ray.experimental.serve.frontend import ZMQClient
from multiprocessing import Process
import time

In [ ]:
zmq_client = ZMQClient(port=8083)

In [ ]:
!ray stop

In [ ]:
def noop():
    return 

In [ ]:
latencies = []
for _ in range(3):
    vals = []
    for i in range(0, 501, 10):
        now = time.perf_counter()
        if i == 0:
            zmq_client.query(1.0, 1000, actor="VectorizedAdder")
        for j in range(i):
            zmq_client.query(1.0, 1000, actor="VectorizedAdder")
        after = time.perf_counter()
        print("Trial", i, "Latency:", after-now)
        vals += [after - now]
    latencies += [vals]

In [ ]:
import numpy as np
latencies = np.array(latencies)
latencies

In [ ]:
avg = np.sum(latencies, axis=0)/np.float(3)
xs = np.array(list(np.arange(0, 501, 10)))

In [ ]:
xs[0] = 1
xs

In [ ]:
import matplotlib.pyplot as plt
plt.plot(xs, avg, 'o-')
plt.title("Latency vs. Number of Requests using ZMQ FrontEnd")
plt.xlabel("Number of Queries")
plt.ylabel("Latency in s")
plt.show()

In [ ]:
np.save("lat_zmq.npy", latencies)

In [ ]:
!ray stop

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import requests

import ray
from ray.experimental.serve import DeadlineAwareRouter
from ray.experimental.serve.examples.adder import VectorizedAdder
from ray.experimental.serve.frontend import HTTPFrontendActor
from ray.experimental.serve.router import start_router

ROUTER_NAME = "DefaultRouter"
ray.shutdown()
ray.init()
router = start_router(DeadlineAwareRouter, ROUTER_NAME)
a = HTTPFrontendActor.remote(router=ROUTER_NAME, port=8091)
a.start.remote()

router.register_actor.remote(
    "VAdder", VectorizedAdder, init_kwargs={"scaler_increment": 1})
latencies = []
url = "http://0.0.0.0:8091/VAdder"
payload = {"input": 10, "slo_ms": 1000}
print("Hello")
time.sleep(10)
print("Done sleeping")
for _ in range(3):
    vals = []
    for i in range(0, 501, 10):
        if i == 0:
            now = time.perf_counter()
            resp = requests.post(url, json=payload)
            assert resp.json() == {"success": True, "actor": "VAdder", "result": 11}
            after = time.perf_counter()
        else:
            now = time.perf_counter()
            for j in range(i):
                resp = requests.request("POST", url, json=payload)
                assert resp.json() == {"success": True, "actor": "VAdder", "result": 11}
            after = time.perf_counter()
        print("Trial", i, "Latency:", after-now)
        vals += [after - now]
    latencies += [vals]

In [ ]:
import numpy as np
latencies = np.array(latencies)
np.save("lat_http.npy", latencies)
latencies

In [ ]:
avg = np.sum(latencies, axis=0)/np.float(3)
xs = np.array(list(np.arange(0, 501, 10)))
xs[0] = 1
xs

In [ ]:
import matplotlib.pyplot as plt
plt.plot(xs, avg, 'o-')
plt.title("Latency vs. Number of Requests using HTTP FrontEnd")
plt.xlabel("Number of Queries")
plt.ylabel("Latency in s")

In [ ]:
times = np.load('lat_zmq.npy')
times

In [ ]:
avgt = np.sum(times, axis=0)/np.float(3)
xs = np.array(list(np.arange(0, 501, 10)))
xs[0] = 1
xs

In [ ]:
plt.plot(xs, avg, 'o-', label='HTTP Frontend')
plt.plot(xs, avgt, '+-', label='ZMQ Frontend')
plt.title("Latency vs. Number of Requests")
plt.xlabel("Number of Queries")
plt.ylabel("Latency in s")
plt.legend()

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import requests

import ray
from ray.experimental.serve import DeadlineAwareRouter
from ray.experimental.serve.examples.test import Predictor
from ray.experimental.serve.router import start_router

ray.shutdown()
ray.init()
handle = start_router(DeadlineAwareRouter, "DefaultRouter")


handle.register_actor.remote(
    "Pred", Predictor, init_kwargs={"container": 'ray/example:latest'})
latencies = []
print("hello")
time.sleep(5)
print("Goodbye")
for _ in range(3):
    vals = []
    for i in range(0, 501, 10):
        if i == 0:
            now = time.perf_counter()
            result = handle.call.remote("Pred", [42], time.perf_counter() + 1)
            after = time.perf_counter()
        else:
            now = time.perf_counter()
            for j in range(i):
                result = handle.call.remote("Pred", [42], time.perf_counter() + 1)
            after = time.perf_counter()
        print("Trial", i, "Latency:", after-now)
        vals += [after - now]
    latencies += [vals]

2019-05-08 01:46:40,098	WARNING worker.py:1344 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-08 01:46:40,099	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-08_01-46-40_9266/logs.
2019-05-08 01:46:40,205	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:24872 to respond...


2019-05-08 01:46:40,324	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:41853 to respond...
2019-05-08 01:46:40,326	INFO services.py:804 -- Starting Redis shard with 6.58 GB max memory.
2019-05-08 01:46:40,334	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-08_01-46-40_9266/logs.
2019-05-08 01:46:40,340	INFO services.py:1439 -- Starting the Plasma object store with 9.87 GB memory using /dev/shm.
2019-05-08 01:46:40,348	WARNING services.py:907 -- Failed to start the reporter. The reporter requires 'pip install psutil'.


hello
(pid=9472) HellO!
(pid=9472) HI!
(pid=9472) GOODBE
(pid=9472) 57033
Goodbye
Trial 0 Latency: 0.0001836039991758298
Trial 10 Latency: 0.0006244930009415839
Trial 20 Latency: 0.0017627389970584773
Trial 30 Latency: 0.002368819001276279
Trial 40 Latency: 0.003286107999883825
Trial 50 Latency: 0.005285195002215914
Trial 60 Latency: 0.006762164000974735
Trial 70 Latency: 0.029583600000478327
Trial 80 Latency: 0.008260350998170907
Trial 90 Latency: 0.008285898999019992
Trial 100 Latency: 0.02979738500289386
Trial 110 Latency: 0.02476829499937594
Trial 120 Latency: 0.01171928899930208
Trial 130 Latency: 0.01241760199991404
Trial 140 Latency: 0.018316637997486396
Trial 150 Latency: 0.013650742002937477
Trial 160 Latency: 0.010003134000726277
Trial 170 Latency: 0.011343525002303068
Trial 180 Latency: 0.010517965001781704
Trial 190 Latency: 0.011973151998972753
Trial 200 Latency: 0.01349786100035999
Trial 210 Latency: 0.012232810000568861
Trial 220 Latency: 0.013560155999584822
Trial 230 L

In [ ]:
latencies = np.array(latencies)
np.save("latencies_cont.npy", latencies)
avg = np.sum(latencies, axis=0)/np.float(3)
xs = np.array(list(np.arange(0, 501, 10)))
xs[0] = 1
xs

In [ ]:
avg

In [ ]:
plt.plot(xs, avg, 'o-')
plt.title("Latency vs. Number of Requests using Containerization")
plt.xlabel("Number of Queries")
plt.ylabel("Latency in s")